In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os, sys

In [ ]:
suffixes_fine_label = [ 'Still_Stand_Outside', 'Still_Stand_Inside',
                  'Still_Sit_Outside', 'Still_Sit_Inside', 'Walking_Outside',
                  'Walking_Inside', 'Run', 'Bike', 'Car_Driver', 'Car_Passenger',
                  'Bus_Stand', 'Bus_Sit', 'Bus_Up_Stand', 'Bus_Up_Sit',
                  'Train_Stand', 'Train_Sit', 'Subway_Stand', 'Subway_Sit']

In [ ]:
labels_car_bus_met = [8, 9, 10, 11, 12, 13, 16, 17]

In [ ]:
def make_output_dir(output_path):
    output_path_split = output_path.split(os.path.sep)
    new_path = '/'
    for directory in output_path_split:
        new_path = os.path.join(new_path, directory)
        if not os.path.exists(new_path):
            os.mkdir(new_path)

In [ ]:
def get_output_file_name(output_path, suffix):
    #  filename = position + '_' + suffix
    filename = suffix
    output_file = os.path.join(output_path, filename + '_01' + '.txt')
    if not os.path.exists(output_file):
        return output_file
    n = 2
    while n < 100:
        if n < 10:
            filename_n = filename + f'_0{n}' + '.txt'
        else:
            filename_n = filename + f'_{n}' + '.txt'
        output_file = os.path.join(output_path, filename_n)
        if not os.path.exists(output_file):
            return output_file
        n += 1

In [ ]:
def extract_data(input_file, label_file):
    f_in = open(input_file, 'r')
    f_label = open(label_file, 'r')

    position = os.path.basename(input_file).split(sep='_')[0]

    while True:
        line_label = f_label.readline()
        if not line_label: break
        line_label_split = line_label.split(sep=' ')
        start = int(line_label_split[0])
        end = int(line_label_split[1])
        label = int(line_label_split[2])

        if label not in labels_car_bus_met: continue

        transport = suffixes_fine_label[label].split(sep='_')[0]
        output_path = os.path.join('/content/drive/Shareddrives/Project/new_sets/train_data/', position, transport)
        print(f'     {output_path}')
        make_output_dir(output_path)
        output_file = get_output_file_name(output_path, suffixes_fine_label[label])
        f_out = open(output_file, 'w')

        while True:
            line_data = f_in.readline()
            if not line_data : break
            line_data_split = line_data.split(sep=' ')
            data_time = int(line_data_split[0].split(sep='.')[0])
            if data_time > end: break #first line gone
            if data_time >= start:
                line_data = ' '.join(line_data_split) + '\n'
                f_out.write(line_data)

        f_out.close()
        if not line_data: break

    f_in.close()
    f_label.close()

In [ ]:
input_filename = 'Hips_Motion.txt'
label_filename = 'labels_track_main.txt'

In [ ]:
for (path, directory, files) in os.walk('/content/drive/Shareddrives/Project/data_original/'):
    for filename in files:
        if filename == input_filename:          
            input_file = os.path.join(path, filename)
            label_file = os.path.join(path, label_filename)
            print(f'input: {input_file}')
            extract_data(input_file, label_file)